<a href="https://colab.research.google.com/github/overred0704/Stock_filter/blob/main/stock_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#import package

In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
import datetime
import time

#set crawler

In [2]:
def crawl_price(date):
    url = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    df = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '}) 
                                        for i in url.text.split('\n') 
                                        if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    df = df.set_index('證券代號')
    df['成交金額'] = df['成交金額'].str.replace(',','')
    df['成交股數'] = df['成交股數'].str.replace(',','')
    return df

#set runing

In [3]:
def running(n_days=3, max_fail_count = 4):
  
  #variable initial
  data = {}
  date = datetime.datetime.now()
  fail_count = 0

  for _ in range(n_days):

    print('loading', date)
    # crawler
    try:
        # start
        data[date.date()] = crawl_price(date)
        print('complete!')
        fail_count = 0
    except:
        # 跳過假日
        print('fail!')
        fail_count += 1
    if fail_count > max_fail_count:
      break
    
    #update day
    date -= datetime.timedelta(days=1)
    time.sleep(20)

  #取得股數
  df = pd.DataFrame({k:d['成交股數'] for k,d in data.items()}).transpose()
  df.index = pd.to_datetime(df.index)
  c = df.transpose()   

  #剔除空值
  c.dropna(axis=0, inplace=True)

  #convert data type and rename columns
  c = c.astype(str).astype(int)
  c.columns = ['今天', '昨天','前天']

  return c

In [4]:
df = running()

loading 2021-06-18 12:30:25.035511
complete!
loading 2021-06-17 12:30:25.035511
complete!
loading 2021-06-16 12:30:25.035511
complete!


In [5]:
w1 = (1/3)
w2 = 3

In [6]:
#calculate threshold
df['threshold1'] = df['今天'] - w1 * df['昨天']
df['threshold2'] = df['前天'] * w2 - df['昨天']

In [7]:
#set filter
f1 = df['threshold1'] < 0
f2 = df['threshold2'] < 0

In [8]:
#write to new df
df_new = df[(f1 & f2 )].sort_values('前天',ascending=False)

df_new = df_new.drop(['threshold1', 'threshold2'], axis=1)

df_new.index.name = None

In [9]:
df_new.to_excel('0618.xlsx')

In [10]:
df_new

,今天,昨天,前天
2406,6629448,43139206,10759430
6477,4415117,21431447,6417341
1809,2420553,11913296,2240984
1454,132063,460565,46387
3266,30000,104001,15000
2891C,7212,33244,7183
4581,7000,47002,6073
1438,11100,37505,2000
1213,0,36341,1014


#filtering function

In [46]:
def filtering(w1 = (1/3), w2 = 3):
  df = running()

  #calculate threshold
  df['threshold1'] = df['今天'] - w1 * df['昨天']
  df['threshold2'] = df['前天'] * w2 - df['昨天']

  #set filter
  f1 = df['threshold1'] < 0
  f2 = df['threshold2'] < 0

  #write to new df
  df_new = df[(f1 & f2 )].sort_values('前天',ascending=False)

  df_new = df_new.drop(['threshold1', 'threshold2'], axis=1)

  df_new.index.name = None

  return df_new

In [47]:
test = filtering()

loading 2021-06-18 09:10:49.389117
complete!
loading 2021-06-17 09:10:49.389117
complete!
loading 2021-06-16 09:10:49.389117
complete!
